In [6]:
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.compat import range
from openpyxl.styles import Font, Color, Border, Side, Alignment, Protection
from openpyxl.utils import get_column_letter
import numpy as np
import pandas as pd
from pandas import DataFrame
import random
import arrow

############ 下面这些需要手工输入 ###########################################

client_name = '刘伟' # 客户姓名
target_race = '厦门 70.3' # 赛事名称
start_date = '2017-02-01' # 开始日期。格式为 YYYY-MM-DD
number_of_weeks = 14 # 周数

race = "half" # 目标赛事长度 sprint, olympic, half, full

recovery = 4  # 每第n周为恢复周

prep_weeks, base_weeks, build_weeks, peak_weeks = (2, 6, 4, 2) # 每个训练阶段持续的周数，括号内数字总数应该等于总周数number_of_weeks

stage_week_dict = {} # 生成每周所属Stage安排的字典
for number_of_week in range(number_of_weeks):
    if number_of_week < prep_weeks:
        stage_week_dict[number_of_week+1] = "prep"
    elif number_of_week >= prep_weeks and number_of_week < (prep_weeks+base_weeks):
        stage_week_dict[number_of_week+1] = "base"
    elif number_of_week >= (prep_weeks+base_weeks) and number_of_week < (prep_weeks+base_weeks+build_weeks):
        stage_week_dict[number_of_week+1] = "build"
    elif number_of_week >= (prep_weeks+base_weeks+build_weeks):
        stage_week_dict[number_of_week+1] = "peak"
        


# 一周7天每天的运动项目和时长安排  ["运动1", "运动2", 时间下限, 时间上限]
'''
运动："swim", "bike", "run", "strength", "rest"
类型: "ae" 有氧耐力, "me" 肌肉耐力, "ss" 速度技巧（限bike）, "fo" 最大力量（限swim和bike）, "bk" 骑跑两项（限bike和run）, "re" 主动恢复

'''
weekly_sport_dict = {1: ["run", "", 0.3, 1.5],       # 周一的安排
                     2: ["bike", "swim", 0.3, 1.5],  # 周二的安排
                     3: ["bike", "run", 0.3, 2],   # 周三的安排
                     4: ["swim", "run", 0.3, 1.5],   # 周四的安排
                     5: ["rest", "", 1, 1],        # 周五的安排
                     6: ["bike", "run", 0.5, 4],     # 周六的安排
                     7: ["swim", "", 0.3, 1.5]     # 周日的安排
                     }

############## 手工输入部分完毕 ##############################################


############## 训练课程数据库 #####################
wko_db_path = "D:/客户训练计划/AutoGenerated/WKO_DB/workout_db.xlsx"
###################################################

# 根据时长, 目标比赛和训练阶段随机选取运动训练课
def get_rand_workout(sport, race, stage, min_d, max_d):
    df = pd.read_excel(wko_db_path, sheetname = sport)
    selected = df[(df['stage'] == stage) & (df['duration'] <= max_d) & (df['duration'] >= min_d) & (df['race'] == race)] # 筛选条件
    rand_index = random.choice(selected.index) # 筛选后随机选择索引
    #print(rand_index)
    #print(selected.at[rand_index, 'workout'])
    return selected.at[rand_index, 'workout'] # 返回随机课程内容

# 生成一天两练的单天总课表
def generate_daily_workout(sport1, race, stage, min_d, max_d, sport2=""):
    if sport1 is 'rest': # 如果是休息日则设置相关几个变量
        stage = 'rest'
        sport2 = ''
        min_d = 1
        max_d = 1
    workout1 = get_rand_workout(sport1, race, stage, min_d, max_d)
    if sport2 is not "":
        workout2 = get_rand_workout(sport2, race, stage, min_d, max_d)
        daily_workout = workout1 + "\n\n" + workout2
    else:
        daily_workout = workout1 + "\n"
    return daily_workout

# 生成训练计划（日历+训练课内容）
def make_race_plan(start_date, number_of_weeks, client_name, target_race):
    #client_name = '王开元' # 客户姓名
    #target_race = '柳州 70.3' # 赛事名称
    #start_date = '2017-02-02' # 开始日期。格式为 YYYY-MM-DD
    #number_of_weeks = 8 # 周数
    
    dest_filename = 'D:/客户训练计划/AutoGenerated/' + client_name + '_' + target_race + '_Training_Plan.xlsx' # 生成Excel文件名

    wb = Workbook()
    ws1 = wb.active
    ws1.title = target_race

    ft_wk = Font(name='Tahoma', size=10, bold=True) # 设置星期的字体
    ft_day = Font(name='Tahoma', size=11) # 设置日期的字体
    alig_wk = Alignment(horizontal='center') # 设置星期的居中
    alig_wko = Alignment(horizontal='general', vertical='top', wrap_text=True) # 设置训练课内容置顶，自动换行
    alig_stg = Alignment(horizontal='center', vertical='top') # 设置Stage居中置顶

    today = arrow.now() # 今天
    start_from = arrow.get(start_date, 'YYYY-MM-DD') # 训练开始第一天

    def day_of_week(day):  # 训练开始第1+day天
        x = start_from.replace(days=+day)
        return x
    #print(end_by.format('dddd'))
    
    first_day = int(start_from.format('d')) # 训练开始第一天是星期几（阿拉伯字母表示）

    #if first_day != 1:  # 如果训练开始的第一天不是星期一，则周数加一周
    #    number_of_weeks += 1 

    row_list_odd = [] # 生成周数的偶数列表，用于填入日期信息
    row_list_even = [] # 生成周数的偶数列表，用于填入训练内容
    for a in range(1, number_of_weeks*2+1):
        if a%2 == 0:
            row_list_even.append(a)
        else:
            row_list_odd.append(a)

    ####### 以下生成日历部分 #######    
    for row, week_number in zip(row_list_odd, range(1, number_of_weeks+1)): # 生成行数和周号
        wk = ws1.cell(column=1, row=row, value='WEEK '+str(week_number)) # 在奇数行的第一列填入周号
        ws1.column_dimensions[get_column_letter(1)].width = 8 # 设置列宽
        wk.font = ft_wk # 设置字体
        wk.alignment = alig_wk # 设置居中
        start = day_of_week(7*week_number-(7+first_day-1)) # 每一周的第一天
        end = day_of_week(7*week_number-(7+first_day-1)+6) # 每一周的第七天
        for col, r in zip(range(2, 9), arrow.Arrow.range('day',start , end)): # 生成列数和对应的日期信息
            date_n_day = r.format('MMM.DD') + ' - '+ r.format('ddd') # 格式化日期信息
            dy = ws1.cell(column=col, row=row, value=date_n_day) # 将日期信息填入列
            ws1.column_dimensions[get_column_letter(col)].width = 22.4 # 设置列宽
            dy.font = ft_day # 设置字体
    
    ####### 以下将训练课填入日历 #######
    workout_rows = [i for i in range((number_of_weeks+1)*2)][2::2] # 生成训练计划内容填入的行数
    
    for i, workout_row in zip(range(number_of_weeks), workout_rows): # 根据总周数循环
        i = i+1 # 第i周
        if i%recovery == 0:
            stage = "recovery"
        else:
            stage = stage_week_dict[i]
        stg = ws1.cell(column=1, row=workout_row, value=stage.title())
        stg.font = ft_day
        stg.alignment = alig_stg
        for a in range(7): # 一周7天循环
            weekday = a+1
            workout_col = a+2
            sport1, sport2, min_d, max_d = weekly_sport_dict[weekday] # 从每周安排中获取运动项目和时长
            if i == 1 and weekday < first_day: # 如果是第1周且日期小于计划开始的第一天
                workout = "" # 留空白
            else:
                workout = generate_daily_workout(sport1, race, stage, min_d, max_d, sport2=sport2) # 生成单日课程内容
            wod = ws1.cell(column=workout_col, row=workout_row,value=workout) # 写入训练课内容
            wod.alignment = alig_wko
            #print(workout + "\n")
               
    try:
        wb.save(filename = dest_filename) # 保存文件
        print("训练计划已生成，共计" + str(number_of_weeks) + "周。")
    except FileNotFoundError:
        Print("文件写入错误！")
    
    #return(number_of_weeks, first_day)

# 执行程序生成训练计划
make_race_plan(start_date, number_of_weeks, client_name, target_race)

训练计划已生成，共计14周。
